테스트 데이터셋 리사이즈 및 복사

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

# ==============================================================================
# 1. 설정
# ==============================================================================
# CSV 파일 경로
CSV_PATH = '/content/drive/MyDrive/REM_project/github/dataset/split_data/master_dataset_verified.csv'

# 원본 이미지 경로 (구글 드라이브)
DRIVE_IMG_ROOT = '/content/drive/MyDrive/mimic_cxr_data/raw'

# 저장할 로컬 경로 (학습 코드와 경로를 맞추기 위해 이름은 유지합니다)
LOCAL_SAVE_DIR = '/content/train_images'
IMG_SIZE = (224, 224)

# ==============================================================================
# 2. 테스트 셋 선별 (학습 때와 동일한 시드로 분리)
# ==============================================================================
print("📂 데이터 로드 및 테스트 셋 분리 중...")
df = pd.read_csv(CSV_PATH)

# ★핵심★: 학습 때와 똑같은 random_state=42를 써야 정확히 테스트 셋만 골라낼 수 있습니다.
subject_ids = df['subject_id'].unique()
train_ids, temp_ids = train_test_split(subject_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

# 테스트 데이터만 필터링
test_df = df[df['subject_id'].isin(test_ids)].reset_index(drop=True)

print(f"✅ 전체 데이터: {len(df)}개")
print(f"🎯 가져올 테스트 데이터: {len(test_df)}개 (전체의 약 10%)")

# ==============================================================================
# 3. 이미지 복사 및 리사이즈 함수
# ==============================================================================
os.makedirs(LOCAL_SAVE_DIR, exist_ok=True)

def process_and_save_test_image(row_path):
    try:
        # 1. 원본 경로
        full_src_path = os.path.join(DRIVE_IMG_ROOT, str(row_path))

        # 2. 저장 경로
        filename = os.path.basename(str(row_path))
        filename = os.path.splitext(filename)[0] + ".jpg"
        full_dst_path = os.path.join(LOCAL_SAVE_DIR, filename)

        # 3. 이미 있으면 패스
        if os.path.exists(full_dst_path):
            return True

        # 4. 리사이즈 및 저장
        with Image.open(full_src_path) as img:
            img = img.convert('RGB')
            img = img.resize(IMG_SIZE, Image.LANCZOS)
            img.save(full_dst_path, "JPEG", quality=85)
        return True

    except Exception as e:
        return False

# ==============================================================================
# 4. 실행 (테스트 셋만 병렬 처리)
# ==============================================================================
paths = test_df['path'].tolist()

print(f"🚀 테스트 셋 이미지 {len(paths)}장을 로컬로 가져옵니다...")

with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(process_and_save_test_image, paths), total=len(paths), desc="Downloading Test Set"))

success_count = sum(results)
print("-" * 50)
print(f"✅ 작업 완료!")
print(f"   - 성공: {success_count}장 / {len(paths)}장")
print(f"   - 저장 위치: {LOCAL_SAVE_DIR}")
print("-" * 50)

모델 정의

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

# 1. 핵심 부품: Gated Cross Attention
# (바이탈이 엑스레이의 7x7 그리드 중 어디가 중요한지 찾아내는 역할)
class GatedCrossAttention(nn.Module):
    def __init__(self, d_model=1024):
        super(GatedCrossAttention, self).__init__()

        # Q, K, V 변환층
        self.query_proj = nn.Linear(d_model, d_model) # Vital (질문자)
        self.key_proj = nn.Linear(d_model, d_model)   # Image (답변 후보)
        self.value_proj = nn.Linear(d_model, d_model) # Image (실제 정보)

        # ★ Gate: 단순한 스칼라 값 (학습 가능한 파라미터)
        # 0에서 시작해서 학습이 될수록 점점 열림
        self.gate = nn.Parameter(torch.tensor(0.0))

        # 후처리
        self.out_proj = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, img_feat, tab_feat):
        # img_feat: (Batch, 1024, 7, 7)
        # tab_feat: (Batch, 1024)
        B, C, H, W = img_feat.shape

        # 1. 차원 맞추기
        img_flat = img_feat.view(B, C, -1).permute(0, 2, 1) # (B, 49, 1024)
        vital_query = tab_feat.unsqueeze(1)                 # (B, 1, 1024)

        # 2. 어텐션 스코어 계산 (여기가 증거를 만드는 곳!)
        Q = self.query_proj(vital_query)
        K = self.key_proj(img_flat)
        V = self.value_proj(img_flat)

        # (Batch, 1, 49) -> 49개 구역에 대한 점수
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (C ** 0.5)
        attn_map = F.softmax(scores, dim=-1)

        # 3. 정보 가져오기 (Context Vector)
        context = torch.matmul(attn_map, V) # (Batch, 1, 1024)

        # 4. Gated Fusion (수도꼭지 역할)
        # 원래 바이탈 정보 + (Gate * 엑스레이에서 가져온 정보)
        gate_value = torch.sigmoid(self.gate)
        fused_feat = vital_query + (gate_value * context)

        fused_feat = self.norm(self.out_proj(fused_feat))

        # 결과값과 함께 'attn_map(증거)'을 리턴해줍니다.
        return fused_feat.squeeze(1), attn_map.view(B, H, W)

# 2. 전체 모델 조립
class DenseNetFusionModel(nn.Module):
    def __init__(self, num_tabular_features, num_classes=1):
        super(DenseNetFusionModel, self).__init__()

        # 이미지 인코더 (DenseNet121)
        # weights='DEFAULT'는 최신 PyTorch 버전용입니다.
        # 구버전이면 pretrained=True로 바꾸세요.
        self.img_enc = models.densenet121(weights='DEFAULT').features

        # 바이탈 인코더 (MLP)
        self.tab_enc = nn.Sequential(
            nn.Linear(num_tabular_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1024), # 이미지 채널수(1024)에 맞춤
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )

        # ★ 교체된 부분: DAFT 대신 Gated Cross Attention 사용
        self.fusion = GatedCrossAttention(d_model=1024)

        # 분류기
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x_img, x_tab):
        img_feat = self.img_enc(x_img)  # (B, 1024, 7, 7)
        tab_feat = self.tab_enc(x_tab)  # (B, 1024)

        # Fusion (여기서 Attention Map이 튀어나옵니다)
        fused_feat, attn_map = self.fusion(img_feat, tab_feat)

        logits = self.classifier(fused_feat)

        # 학습할 때는 logits만 필요하지만, 나중에 그림 그리려고 attn_map도 같이 뱉습니다.
        return logits, attn_map

데이터셋 정의

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# 1. 설정
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'
DATA_PATH = BASE_DIR + 'master_dataset_verified.csv'
IMG_ROOT = '/content/train_images/'

# 2. 데이터 로드 및 분할 (학습 때와 동일한 Seed=42 필수)
print("📂 데이터 로드 및 분할 중...")
df = pd.read_csv(DATA_PATH)
subject_ids = df['subject_id'].unique()

train_ids, temp_ids = train_test_split(subject_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

# 테스트 데이터셋만 필요
test_df = df[df['subject_id'].isin(test_ids)].reset_index(drop=True)

# 3. 정규화용 통계량 (Train 셋 기준값 - 학습 때 쓴 값과 동일해야 함)
# 여기서는 편의상 전체 데이터 기준으로 근사값을 사용하거나, 학습 코드의 값을 가져옵니다.
# 현재 코드 (Line 194)
TABULAR_COLS = ['gender', 'anchor_age', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'WBC', 'BNP']# (원래는 train_df에서 계산해야 하지만, 실행 편의를 위해 여기선 test_df 값으로 대체하거나 하드코딩 가능)
# 정확도를 위해 Train 셋을 잠깐 불러와서 계산
train_subset = df[df['subject_id'].isin(train_ids)]
scaler_mean = train_subset[TABULAR_COLS].mean()
scaler_std = train_subset[TABULAR_COLS].std().replace(0, 1)

# 4. Dataset 클래스 정의
class MultimodalDataset(Dataset):
    def __init__(self, dataframe, img_root, transform=None):
        self.df = dataframe
        self.img_root = img_root
        self.transform = transform
        self.target_cols = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
        self.tabular_cols = TABULAR_COLS
        self.mean = scaler_mean
        self.std = scaler_std

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # 이미지
        original_path = str(row['path'])
        filename = os.path.basename(original_path)
        filename = os.path.splitext(filename)[0] + ".jpg"
        full_img_path = os.path.join(self.img_root, filename)

        try:
            image = Image.open(full_img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), (0, 0, 0)) # 에러 방지용 검은색 이미지

        if self.transform:
            image = self.transform(image)

        # 바이탈 데이터 (정규화)
        tabular = row[self.tabular_cols].values.astype(np.float32)
        tabular = (tabular - self.mean.values.astype(np.float32)) / self.std.values.astype(np.float32)

        # 정답 라벨
        labels = row[self.target_cols].values.astype(np.float32)

        return image, torch.tensor(tabular), torch.tensor(labels)

# 5. Loader 생성
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = MultimodalDataset(test_df, IMG_ROOT, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Make TARGET_COLS globally available for other cells
TARGET_COLS = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']

print(f"✅ test_loader 준비 완료! (데이터 개수: {len(test_dataset)}개)")

퓨전 모델과 이미지 온리 모델 비교 테스트 시작

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, roc_auc_score # Added roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

# Assuming BASE_DIR is defined in a previous cell
# If not, you might need to define it here or run the cell where it's defined.
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'

# ---------------------------------------------------------
# 1. Image-Only 모델 클래스 정의 (비교군)
# ---------------------------------------------------------
class ImageOnlyDenseNet(nn.Module):
    def __init__(self, num_classes=6): # Changed num_classes to 6
        super().__init__()
        # Fusion 모델과 공정한 비교를 위해 똑같은 DenseNet121 사용
        self.backbone = models.densenet121(pretrained=False)
        num_ftrs = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.backbone(x)

# ---------------------------------------------------------
# 2. 평가 함수 (Fusion vs Image-Only 자동 구분)
# ---------------------------------------------------------
def get_all_predictions(model, loader, device, model_type='fusion'):
    model.eval()
    y_true_list = []
    y_scores_list = [] # Store probabilities for all classes
    y_preds_list = []  # Store 0 or 1 predictions for all classes

    with torch.no_grad():
        for images, tabular, labels in tqdm(loader, desc=f"Testing {model_type}..."):
            images = images.to(device).float()
            labels = labels.to(device).float()

            # 모델 타입에 따라 입력 방식 분기 처리
            if model_type == 'fusion':
                tabular = tabular.to(device).float()
                logits, _ = model(images, tabular) # Extract logits from the tuple output
            else:
                logits = model(images)

            probs = torch.sigmoid(logits).cpu().numpy() # Apply sigmoid only to logits
            preds = (probs > 0.5).astype(int)            # (batch_size, num_classes)

            y_true_list.append(labels.cpu().numpy())
            y_scores_list.append(probs)
            y_preds_list.append(preds)

    return np.concatenate(y_true_list), np.concatenate(y_scores_list), np.concatenate(y_preds_list)

# ---------------------------------------------------------
# 3. 메인 비교 실행 (Main Execution)
# ---------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (1) 데이터 로더 준비 (test_loader가 이미 정의되어 있다고 가정)
# test_loader = ... (선생님의 test_loader 사용)

# (2) Fusion 모델 로드
fusion_model = DenseNetFusionModel(num_classes=6, num_tabular_features=10).to(device) # Changed tabular_input_dim to 10
# 저장해둔 Fusion 가중치 불러오기
try:
    # Corrected path to use BASE_DIR
    fusion_model.load_state_dict(torch.load(os.path.join(BASE_DIR, "best_fusion_model.pth")))
    print("✅ Fusion Model Loaded.")
except FileNotFoundError:
    print("⚠️ Fusion Model weights not found. Skipping...")
except RuntimeError as e:
    print(f"❌ Error loading Fusion Model weights: {e}. Skipping...")

# (3) Image-Only 모델 로드 (비교군)
image_model = ImageOnlyDenseNet(num_classes=6).to(device) # Changed num_classes to 6
# 저장해둔 Image-Only 가중치 불러오기
try:
    # Corrected path to use BASE_DIR
    image_model.load_state_dict(torch.load(os.path.join(BASE_DIR, "best_image_only_model.pth")))
    print("✅ Image-Only Model Loaded.")
except FileNotFoundError:
    print("⚠️ Image-Only Model weights not found. (비교를 위해 학습 후 저장 필요)")
except RuntimeError as e:
    print(f"❌ Error loading Image-Only Model weights: {e}. Skipping...")

# (4) 추론 실행
print("\n--- Running Inference ---")
# Fusion 예측
y_true_all, f_scores_all, f_preds_all = get_all_predictions(fusion_model, test_loader, device, model_type='fusion')
# Image-Only 예측
_, i_scores_all, i_preds_all = get_all_predictions(image_model, test_loader, device, model_type='image_only')

num_classes = y_true_all.shape[1] # Number of diseases

# Calculate Macro AUROC
f_aucs_macro = []
i_aucs_macro = []
for i in range(num_classes):
    try:
        f_aucs_macro.append(roc_auc_score(y_true_all[:, i], f_scores_all[:, i]))
    except ValueError:
        f_aucs_macro.append(0.5) # Handle cases with only one class present
    try:
        i_aucs_macro.append(roc_auc_score(y_true_all[:, i], i_scores_all[:, i]))
    except ValueError:
        i_aucs_macro.append(0.5)

f_auc_macro = np.mean(f_aucs_macro)
i_auc_macro = np.mean(i_aucs_macro)

# Calculate Micro AUROC for plotting and overall summary
f_fpr_micro, f_tpr_micro, _ = roc_curve(y_true_all.ravel(), f_scores_all.ravel())
f_auc_micro = auc(f_fpr_micro, f_tpr_micro)

i_fpr_micro, i_tpr_micro, _ = roc_curve(y_true_all.ravel(), i_scores_all.ravel())
i_auc_micro = auc(i_fpr_micro, i_tpr_micro)

print(f"\n📊 Result Summary:")
print(f"Image-Only Macro AUROC: {i_auc_macro:.4f}")
print(f"Fusion     Macro AUROC: {f_auc_macro:.4f} (Diff: {f_auc_macro - i_auc_macro:+.4f})")
print(f"\nImage-Only Micro AUROC: {i_auc_micro:.4f}")
print(f"Fusion     Micro AUROC: {f_auc_micro:.4f} (Diff: {f_auc_micro - i_auc_micro:+.4f})")

# ---------------------------------------------------------
# 4. 시각화: ROC Curve 겹쳐 그리기 (논문용 그림) - Using Micro-Average
# ---------------------------------------------------------
plt.figure(figsize=(8, 6))

# Image-Only 곡선 (파선)
plt.plot(i_fpr_micro, i_tpr_micro, linestyle='--', color='gray', label=f'Image Only (Micro AUC = {i_auc_micro:.3f})')

# Fusion 곡선 (실선, 빨간색 강조)
plt.plot(f_fpr_micro, f_tpr_micro, linestyle='-', color='red', linewidth=2, label=f'Ours (Fusion) (Micro AUC = {f_auc_micro:.3f})')

# 기준선
plt.plot([0, 1], [0, 1], color='navy', linestyle=':', alpha=0.5)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison: Fusion vs Image-Only (Micro-Average)')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

# ---------------------------------------------------------
# 5. 혼동 행렬 (Confusion Matrix) 비교 - 선택 사항 (Micro-Average)
# ---------------------------------------------------------
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Flatten for micro-average confusion matrix and accuracy
sns.heatmap(confusion_matrix(y_true_all.flatten(), i_preds_all.flatten()), annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title(f"Image-Only (Micro Acc: {accuracy_score(y_true_all.flatten(), i_preds_all.flatten()):.3f})")
ax[0].set_xlabel("Predicted")
ax[0].set_ylabel("Actual")

sns.heatmap(confusion_matrix(y_true_all.flatten(), f_preds_all.flatten()), annot=True, fmt='d', cmap='Reds', ax=ax[1])
ax[1].set_title(f"Fusion (Micro Acc: {accuracy_score(y_true_all.flatten(), f_preds_all.flatten()):.3f})")
ax[1].set_xlabel("Predicted")
ax[1].set_ylabel("Actual")

plt.tight_layout()
plt.show()

아래는 논문에 넣기 위한 표와 그래프를 뽑는 코드들입니다...

In [ ]:
# -----------------------------------------------------------
# [Fix] 변수명 매핑 (이름표 달아주기)
# -----------------------------------------------------------
# 앞선 코드에서 로드한 모델들을 진단 코드의 이름으로 연결합니다.
model = fusion_model           # Multimodal 모델
model_img_only = image_model   # Image-Only 모델

print("✅ 모델 변수명 연결 완료!")
print(f"   - model -> {type(model).__name__}")
print(f"   - model_img_only -> {type(model_img_only).__name__}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

# -----------------------------------------------------------------------------
# 1. 설정
# -----------------------------------------------------------------------------
# 분석할 6대 질환명 (순서가 데이터셋과 일치해야 함)
DISEASE_NAMES = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']

# -----------------------------------------------------------------------------
# 2. 질환별 AUROC 계산 함수
# -----------------------------------------------------------------------------
def analyze_per_disease(model_fusion, model_img, loader, device):
    print("🔍 질환별 성능 정밀 분석 중... (잠시만 기다려주세요)")

    # 1. 예측값 추출
    y_true, f_scores, _ = get_all_predictions(model_fusion, loader, device, model_type='fusion')
    _, i_scores, _ = get_all_predictions(model_img, loader, device, model_type='image_only')

    # y_true, f_scores의 shape은 (N_samples, 6) 이어야 함 (Flatten 된 상태면 Reshape 필요)
    # get_all_predictions 함수가 flatten을 반환했다면 다시 reshape
    if f_scores.ndim == 1:
        f_scores = f_scores.reshape(-1, 6)
        i_scores = i_scores.reshape(-1, 6)
        y_true = y_true.reshape(-1, 6)

    results = []

    # 2. 질환별 계산
    for idx, disease in enumerate(DISEASE_NAMES):
        # 정답이 하나도 없는 클래스 방지
        if len(np.unique(y_true[:, idx])) < 2:
            print(f"⚠️ {disease}: 테스트셋에 양성 샘플이 없어서 건너뜁니다.")
            continue

        auc_fusion = roc_auc_score(y_true[:, idx], f_scores[:, idx])
        auc_img = roc_auc_score(y_true[:, idx], i_scores[:, idx])

        diff = auc_fusion - auc_img
        status = "🔺상승" if diff > 0 else "🔻하락"

        results.append({
            "Disease": disease,
            "Image-Only": auc_img,
            "Fusion (Ours)": auc_fusion,
            "Diff": diff,
            "Status": status
        })

    return pd.DataFrame(results)

# -----------------------------------------------------------------------------
# 3. 실행 및 시각화
# -----------------------------------------------------------------------------
# 모델과 로더가 메모리에 있어야 합니다.
df_result = analyze_per_disease(fusion_model, image_model, test_loader, device)

# 4. 표 출력 (예쁘게)
print("\n" + "="*60)
print("📊 [질환별 성능 상세 비교표]")
print("="*60)
# 소수점 4자리까지 표시
print(df_result.round(4).to_string(index=False))

# 5. 그래프 시각화 (상승/하락폭)
plt.figure(figsize=(10, 6))
colors = ['red' if x > 0 else 'blue' for x in df_result['Diff']]
sns.barplot(x='Diff', y='Disease', data=df_result, palette=colors)

plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.title(f"Performance Gain by Fusion (Total Mean Diff: {df_result['Diff'].mean():.4f})")
plt.xlabel("AUROC Improvement (Fusion - ImageOnly)")
plt.grid(axis='x', alpha=0.3)

# 수치 표시
for i, v in enumerate(df_result['Diff']):
    plt.text(v, i, f"{v:+.4f}", va='center', fontweight='bold',
             color='red' if v > 0 else 'blue')

plt.show()

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# =============================================================================
# 1. 설정 (이전 코드 실행 후 변수들이 메모리에 있다고 가정)
# =============================================================================
# 만약 이전 코드가 꺼졌다면, 03번 코드의 모델 로드 부분까지 다시 실행해주세요.
print("🧪 바이탈 의존도(Permutation Importance) 테스트 시작...")

def get_auc_per_disease(labels, preds):
    aucs = []
    for i in range(len(TARGET_COLS)):
        try:
            score = roc_auc_score(labels[:, i], preds[:, i])
        except:
            score = 0.5
        aucs.append(score)
    return np.mean(aucs), aucs

# =============================================================================
# 2. 정상 데이터 추론 (Baseline)
# =============================================================================
# 이미 03번 코드에서 preds_multi를 구했다면 생략 가능하지만, 확실히 하기 위해 다시 함
model.eval() # Changed model_multi to model
all_preds_original = []
all_labels = []

with torch.no_grad():
    for imgs, tabs, lbls in tqdm(test_loader, desc="Original Inference"):
        imgs, tabs = imgs.to(device), tabs.to(device)
        outputs = model(imgs, tabs)
        logits, _ = outputs # Extract logits from the tuple output
        all_preds_original.append(torch.sigmoid(logits).cpu().numpy())
        all_labels.append(lbls.numpy())

preds_original = np.vstack(all_preds_original)
labels_np = np.vstack(all_labels)
mean_auc_orig, aucs_orig = get_auc_per_disease(labels_np, preds_original)

print(f"\n✅ 원본 평균 AUROC: {mean_auc_orig:.4f}")

# =============================================================================
# 3. 바이탈 셔플링 추론 (Shuffled Inference)
# =============================================================================
# 데이터로더를 다시 돌면서, 이번엔 tabular 데이터만 배치 내에서 섞어버림
all_preds_shuffled = []

with torch.no_grad():
    for imgs, tabs, lbls in tqdm(test_loader, desc="Shuffled Vitals"):
        imgs, tabs = imgs.to(device), tabs.to(device)

        # ★ 핵심: 바이탈 데이터(tabs)를 랜덤하게 섞어버림 (Batch 내 셔플)
        # 환자 A의 사진에 환자 B의 바이탈이 들어가는 상황을 만듦
        shuffled_idx = torch.randperm(tabs.size(0))
        tabs_shuffled = tabs[shuffled_idx]

        outputs = model(imgs, tabs_shuffled)
        logits, _ = outputs # Extract logits from the tuple output
        all_preds_shuffled.append(torch.sigmoid(logits).cpu().numpy())

preds_shuffled = np.vstack(all_preds_shuffled)
mean_auc_shuf, aucs_shuf = get_auc_per_disease(labels_np, preds_shuffled)

print(f"❌ 셔플 평균 AUROC: {mean_auc_shuf:.4f}")

# =============================================================================
# 4. 결과 비교 및 시각화
# =============================================================================
drop = mean_auc_orig - mean_auc_shuf
print(f"\n📉 성능 하락폭 (Importance): {drop:.4f}")

if drop < 0.005:
    print("🚨 결론: 모델이 바이탈을 거의 무시하고 있습니다! (점수 차이 없음)")
else:
    print("🟢 결론: 모델이 바이탈 정보를 유의미하게 사용하고 있습니다.")

# 질환별 하락폭 시각화
drops = [a - b for a, b in zip(aucs_orig, aucs_shuf)]
plt.figure(figsize=(10, 6))
plt.bar(TARGET_COLS, drops, color='#ff6b6b', edgecolor='black')
plt.title("Performance Drop when Vitals are Randomized", fontsize=14)
plt.ylabel("AUROC Drop")
plt.grid(axis='y', alpha=0.3)
for i, v in enumerate(drops):
    plt.text(i, v, f"{v:.4f}", ha='center', va='bottom')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# ---------------------------------------------------------
# 1. 설정
# ---------------------------------------------------------
DISEASE_NAMES = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
THRESHOLD = 0.5 # 0.5 이상이면 양성으로 판단

# ---------------------------------------------------------
# 2. 상세 지표 계산 함수
# ---------------------------------------------------------
def get_detailed_metrics(y_true, y_scores):
    # 확률 -> 0/1 예측값 변환
    y_pred = (y_scores > THRESHOLD).astype(int)

    # 혼동 행렬 (TN, FP, FN, TP)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # 민감도 (Recall, Sensitivity): 환자를 환자라고 맞춘 비율
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0

    # 특이도 (Specificity): 정상을 정상이라고 맞춘 비율
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # 정밀도 (Precision)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    return sensitivity, specificity, precision, tp, fn

# ---------------------------------------------------------
# 3. 분석 실행
# ---------------------------------------------------------
# 예측값 가져오기 (이미 메모리에 있다면 생략 가능)
print("running analysis...")
y_true, f_scores, _ = get_all_predictions(fusion_model, test_loader, device, model_type='fusion')
_, i_scores, _ = get_all_predictions(image_model, test_loader, device, model_type='image_only')

# Shape 맞추기
if f_scores.ndim == 1:
    f_scores = f_scores.reshape(-1, 6)
    i_scores = i_scores.reshape(-1, 6)
    y_true = y_true.reshape(-1, 6)

results = []

for idx, disease in enumerate(DISEASE_NAMES):
    # Image-Only 지표
    i_sens, i_spec, i_prec, i_tp, i_fn = get_detailed_metrics(y_true[:, idx], i_scores[:, idx])

    # Fusion 지표
    f_sens, f_spec, f_prec, f_tp, f_fn = get_detailed_metrics(y_true[:, idx], f_scores[:, idx])

    # 변화량 (Fusion - ImageOnly)
    diff_sens = f_sens - i_sens

    results.append({
        "Disease": disease,
        "Img_Sens": f"{i_sens:.4f}",
        "Fus_Sens": f"{f_sens:.4f}",
        "Diff_Sens": f"{diff_sens:+.4f}", # 민감도 변화 (중요!)
        "Img_Missed(FN)": i_fn,
        "Fus_Missed(FN)": f_fn, # 놓친 환자 수 (줄어들면 좋음)
        "Gain(TP)": f_tp - i_tp # 추가로 찾아낸 환자 수
    })

# ---------------------------------------------------------
# 4. 결과 출력
# ---------------------------------------------------------
df_analysis = pd.DataFrame(results)
print("\n🏥 [질환별 상세 분석 리포트]")
print("Focus: 민감도(Sensitivity)가 오른 질병을 찾으세요!")
print("-" * 80)
print(df_analysis.to_string(index=False))
print("-" * 80)

히트맵 시각화...

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2

# -------------------------------------------------------------------------
# 함수 정의: Image Only(CNN Feature) vs Fusion(Attention) 비교
# -------------------------------------------------------------------------
def compare_visual_evidence(model, dataloader, target_disease='Pneumonia', num_samples=3):

    # 클래스 이름 (선생님 데이터셋 순서에 맞게 수정 필요)
    class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
    target_idx = class_names.index(target_disease)

    print(f"🔎 [{target_disease}] 비교 분석 시작...")
    model.eval()

    found = 0
    with torch.no_grad():
        for imgs, vitals, labels in dataloader:
            imgs, vitals, labels = imgs.to(device), vitals.to(device), labels.to(device)

            # 1. 모델에서 특징과 어텐션 맵 추출
            # (DenseNetFusionModel의 forward를 수정했으므로 img_feat 접근 가능하도록 호출)

            # 이미지 인코더의 순수 특징 (Image Only 시선 대용)
            img_feat = model.img_enc(imgs)  # (B, 1024, 7, 7)

            # Fusion 모델의 예측 및 어텐션 맵
            logits, attn_maps = model(imgs, vitals)
            probs = torch.sigmoid(logits)

            for i in range(imgs.size(0)):
                # 조건: 실제 환자이고(Target=1), 모델이 강하게 예측(Prob>0.7)한 경우
                if labels[i, target_idx] == 1 and probs[i, target_idx] > 0.7:

                    # --- A. 원본 이미지 복원 ---
                    inv_img = imgs[i].cpu().permute(1, 2, 0).numpy()
                    inv_img = (inv_img - inv_img.min()) / (inv_img.max() - inv_img.min())
                    inv_img_uint8 = np.uint8(255 * inv_img)

                    # --- B. Image Only 시선 (CNN Feature Map 평균) ---
                    # 1024개 채널의 활성화를 평균내어 "CNN이 중요하게 본 곳"을 추정
                    cam_img = img_feat[i].mean(dim=0).cpu().numpy() # (7, 7)
                    cam_img = cv2.resize(cam_img, (224, 224))
                    cam_img = (cam_img - cam_img.min()) / (cam_img.max() - cam_img.min())
                    heatmap_img = cv2.applyColorMap(np.uint8(255 * cam_img), cv2.COLORMAP_VIRIDIS)
                    heatmap_img = cv2.cvtColor(heatmap_img, cv2.COLOR_BGR2RGB)

                    # --- C. Fusion 시선 (Gated Cross Attention Map) ---
                    # 바이탈 정보가 가중치를 부여한 "진짜 증거"
                    att = attn_maps[i].cpu().numpy() # (7, 7)
                    att = cv2.resize(att, (224, 224))
                    att = (att - att.min()) / (att.max() - att.min()) # 정규화
                    heatmap_fus = cv2.applyColorMap(np.uint8(255 * att), cv2.COLORMAP_JET) # 붉은색 강조
                    heatmap_fus = cv2.cvtColor(heatmap_fus, cv2.COLOR_BGR2RGB)

                    # --- D. 오버레이 생성 ---
                    overlay_img = cv2.addWeighted(inv_img_uint8, 0.6, heatmap_img, 0.4, 0)
                    overlay_fus = cv2.addWeighted(inv_img_uint8, 0.6, heatmap_fus, 0.4, 0)

                    # --- 시각화 ---
                    plt.figure(figsize=(15, 5))

                    # 1. 원본
                    plt.subplot(1, 4, 1)
                    plt.title(f"Original X-ray\n({target_disease})")
                    plt.imshow(inv_img)
                    plt.axis('off')

                    # 2. Image Only (CNN 시선)
                    plt.subplot(1, 4, 2)
                    plt.title("Image Only Focus\n(CNN Features)")
                    plt.imshow(overlay_img)
                    plt.axis('off')

                    # 3. Fusion (Vital Guide)
                    plt.subplot(1, 4, 3)
                    plt.title("Fusion Focus\n(Vital Guided)")
                    plt.imshow(overlay_fus) # 붉은색이 집중된 곳
                    plt.axis('off')

                    # 4. 차이 강조 (Fusion - Image)
                    # 퓨전이 더 보게 된 곳을 표시
                    diff = att - cam_img
                    diff = np.maximum(diff, 0) # 양수만 (퓨전 덕분에 더 보게 된 곳)
                    diff = (diff - diff.min()) / (diff.max() - diff.min() + 1e-8)
                    heatmap_diff = cv2.applyColorMap(np.uint8(255 * diff), cv2.COLORMAP_HOT)
                    heatmap_diff = cv2.cvtColor(heatmap_diff, cv2.COLOR_BGR2RGB)

                    plt.subplot(1, 4, 4)
                    plt.title("Vital Gain\n(Added Attention)")
                    plt.imshow(heatmap_diff)
                    plt.axis('off')

                    plt.tight_layout()
                    plt.show()

                    found += 1
                    if found >= num_samples:
                        print("✅ 시각화 완료.")
                        return

# 실행 (test_loader 사용)
compare_visual_evidence(model, test_loader, target_disease='Pneumonia', num_samples=3)

In [ ]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

def save_original_resolution_heatmap(model, loader, dataframe, target_disease='Pneumonia', save_dir='./', num_samples=1):
    # 1. 설정
    class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
    target_idx = class_names.index(target_disease)

    # 원본 이미지 폴더 경로 (설정된 경로 활용)
    # 03번 파일 상단에 정의된 DRIVE_IMG_ROOT 사용
    img_root_path = '/content/drive/MyDrive/mimic_cxr_data/raw'

    model.eval()
    print(f"qh 📸 [{target_disease}] 고화질 원본 경로 매칭 및 히트맵 생성 중...")

    # 전체 데이터를 순회하며 찾기
    global_idx = 0
    found_count = 0

    with torch.no_grad():
        for imgs, vitals, labels in loader:
            imgs = imgs.to(device)
            vitals = vitals.to(device)

            # Fusion 모델 예측 및 어텐션 맵 추출
            logits, attn_maps = model(imgs, vitals) # Fusion 모델은 vitals도 받음
            probs = torch.sigmoid(logits)

            for i in range(imgs.size(0)):
                # 조건: 실제 환자이고(1), 모델 예측 확률이 80% 이상인 경우
                if labels[i, target_idx] == 1 and probs[i, target_idx] > 0.8:

                    # -----------------------------------------------------------
                    # 1. 원본 파일 경로 찾기 (DataFrame 이용)
                    # -----------------------------------------------------------
                    # current_idx 계산 로직 변경: global_idx는 배치 시작 인덱스, i는 배치 내 인덱스
                    # test_loader의 shuffle이 False이므로, df 인덱스와 global_idx + i가 일치해야 함
                    row = dataframe.iloc[global_idx + i]
                    original_file_path = os.path.join(img_root_path, str(row['path']))

                    # 원본 이미지 로드 (고화질)
                    if not os.path.exists(original_file_path):
                        print(f"⚠️ 원본 파일을 찾을 수 없음: {original_file_path}")
                        continue

                    # cv2로 고화질 읽기 (BGR)
                    src_img = cv2.imread(original_file_path)
                    if src_img is None:
                        print(f"⚠️ 이미지를 읽을 수 없음: {original_file_path}")
                        continue

                    # 시각화를 위해 RGB로 변환
                    src_img_rgb = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
                    h, w, _ = src_img.shape # 원본 해상도 (예: 2500x2000 등)

                    print(f"✅ 발견! Index: {global_idx + i}")
                    print(f"   - 파일 경로: {original_file_path}")
                    print(f"   - 원본 해상도: {w} x {h}, 예측확률: {probs[i, target_idx]:.4f}")

                    # -----------------------------------------------------------
                    # 2. 히트맵 생성 (7x7 -> 원본 크기로 확대)
                    # -----------------------------------------------------------
                    # Fusion 모델의 attn_map 사용
                    cam_small = attn_maps[i].cpu().numpy() # (7, 7) 형태

                    # 정규화 (0~1)
                    cam_norm = (cam_small - cam_small.min()) / (cam_small.max() - cam_small.min() + 1e-8)

                    # 원본 크기(w, h)로 확대 (이게 핵심!)
                    cam_resized = cv2.resize(cam_norm, (w, h))

                    # 컬러맵 적용 (0~255 uint8 변환 후 적용) - JET 컬러맵이 더 명확할 수 있음
                    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)

                    # -----------------------------------------------------------
                    # 3. 원본과 합치기
                    # -----------------------------------------------------------
                    # 원본(src_img)에 히트맵을 50:50으로 섞기
                    overlay = cv2.addWeighted(src_img, 0.6, heatmap, 0.4, 0)

                    # -----------------------------------------------------------
                    # 4. 저장 및 출력
                    # -----------------------------------------------------------
                    # 오버레이 이미지 저장
                    save_name_overlay = f"{save_dir}HQ_Fusion_Overlay_{target_disease}_{global_idx + i}.jpg"
                    cv2.imwrite(save_name_overlay, overlay)
                    print(f"💾 고화질 오버레이 저장 완료: {save_name_overlay}")

                    # 원본 이미지 저장
                    save_name_original = f"{save_dir}HQ_Original_Image_{target_disease}_{global_idx + i}.jpg"
                    cv2.imwrite(save_name_original, src_img)
                    print(f"💾 고화질 원본 이미지 저장 완료: {save_name_original}")

                    # 노트북 상에서 확인용 (작게 출력)
                    plt.figure(figsize=(10, 5))
                    plt.subplot(1, 2, 1)
                    plt.imshow(src_img_rgb)
                    plt.title("Original High-Res")
                    plt.axis('off')

                    plt.subplot(1, 2, 2)
                    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
                    plt.title(f"Fusion Model Heatmap\n(Prob: {probs[i, target_idx]:.2f})")
                    plt.axis('off')
                    plt.show()

                    found_count += 1
                    if found_count >= num_samples:
                        return # 원하는 샘플 수만큼 찾으면 종료
            global_idx += imgs.size(0)

# 실행 (Image Only 모델 사용) -> Fusion 모델로 변경, 10개 샘플 추출
# test_df는 03번 파일 상단에서 로드된 데이터프레임입니다.
save_original_resolution_heatmap(model, test_loader, test_df, target_disease='Pneumonia', num_samples=10)
